<a href="https://colab.research.google.com/github/rakshitha2207/Vector-store---Embedding-model/blob/main/vector_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install required libraries

In [ ]:
!pip install -q -U \
     Sentence-transformers==3.0.1 \
     langchain==0.2.11 \
     langchain-huggingface==0.0.3 \
     langchain-google-genai==1.0.7 \
     langchain-chroma==0.1.2 \
     langchain-community==0.2.10 \
     einops==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00


# Install required libraries

In [ ]:
import os
import getpass
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from sklearn.metrics.pairwise import cosine_similarity

# Provide Google API Key.

It can be used both for Gemini Pro LLM  & Google Embedding Model. You can create Google API key using following link

1. [Google Gemini-Pro API Key](https://console.cloud.google.com/apis/credentials)

2. [YouTube Video explaining Google API Key](https://www.youtube.com/watch?v=ZHX7zxvDfoc)



In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


# Provide Huggingface API Key.

If you want to use Huggingface Embedding Models. You can create Huggingface API key using following link

1. [Huggingface API Key](https://huggingface.co/settings/tokens)




In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


# Huggingface based Nomic AI Embedding model

You can use any other huggingface open-source embedding models as per your requirement, fitness and system constraints. You can get the model name from MTEB leaderboard.

Popular Models

1. nomic-ai/nomic-embed-text-v1.5
2. nomic-ai/nomic-embed-text-v1
3. sentence-transformers/all-MiniLM-L12-v2
4. sentence-transformers/all-MiniLM-L6-v2

.....



In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

def get_huggingface_embeddings():

    # Change model_name as per your choosen huggingface embedding model
    nomic_embeddings = HuggingFaceEmbeddings(model_name="nomic-ai/nomic-embed-text-v1.5", model_kwargs = {'trust_remote_code': True})
    return nomic_embeddings

# Let's implement Basic RAG using following components

1. **Chroma:** Used as the vector store for efficient similarity search.
2. **Embedding Models:** As we choose OpenAI, Google or Huggingface Embedding Models
3. **ChatGoogleGenerativeAI:** The Gemini Pro model used for generating responses.
4. **cosine_similarity:** Used for computing similarities in the evaluation step.

In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    # Iterate through each document and format the output
    for i, d in enumerate(docs):
        print(f"{'-' * 50}\nDocument {i + 1}:")
        print(f"Content:\n{d.page_content}\n")
        print("Metadata:")
        for key, value in d.metadata.items():
            print(f"  {key}: {value}")
    print(f"{'-' * 50}")  # Final separator for clarity

# Example usage
# Assuming `docs` is a list of Document objects

# Step 1: Load and preprocess data code

In [ ]:
def load_and_process_data(url):
    # Load data from web
    loader = WebBaseLoader(url)
    data = loader.load()

    # Split text into chunks (Experiment with Chunk Size and Chunk Overlap to get optimal chunking)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = text_splitter.split_documents(data)

    # Add unique IDs to each text chunk
    for idx, chunk in enumerate(chunks):
        chunk.metadata["id"] = idx

    return chunks

# Step 2: Intialize Embedding Model

In [ ]:
def get_embedding_model():
        embeddings = get_huggingface_embeddings()
        return embeddings

# Step 3: Create Chroma vector store

In [ ]:
def create_vector_stores(chunks, embeddings):
    # Create vector stores using the specified embedding model
    vectorstore = Chroma.from_documents(chunks, embeddings)
    return vectorstore

# Step 4: Implement Basic RAG

1. **Retrieval:** We use the Chroma vector store's similarity search to retrieve the top 5 most relevant documents for the query.
2. **Context Formation:** We combine the retrieved documents into a single context string.
3. **Response Generation:** Using the Gemini Pro model, we generate a final response based on the retrieved context and the query.

In [ ]:
def basic_rag(query, vectorstore, llm):
    # Retrieve relevant documents
    docs = vectorstore.similarity_search(query, k=5)
    context = "\n\n".join([doc.page_content for doc in docs])

    # Generate response
    prompt = f"{context}\n\nQuestion: {query}\nAnswer:"
    response = llm.invoke(prompt)

    return {
        "query": query,
        "final_answer": response.content,
        "retrieval_method": "Basic similarity search",
        "context": context
    }

# Step 5: RAG Evaluation

1. We compute embeddings for the query, response, and context.
2. We calculate cosine similarities between query-response and response-context.
3. We compute an overall relevance score as the average of these similarities.


In [ ]:
def evaluate_response(query, embeddings, response, context):
    # Compute embeddings
    query_embedding = embeddings.embed_query(query)
    response_embedding = embeddings.embed_query(response)
    context_embedding = embeddings.embed_query(context)

    # Compute cosine similarities
    query_response_similarity = cosine_similarity([query_embedding], [response_embedding])[0][0]
    response_context_similarity = cosine_similarity([response_embedding], [context_embedding])[0][0]

    # Compute relevance score (average of the two similarities)
    relevance_score = (query_response_similarity + response_context_similarity) / 2

    return {
        "query_response_similarity": query_response_similarity,
        "response_context_similarity": response_context_similarity,
        "relevance_score": relevance_score
    }

# Step 4: Load and process data and create chunks to store in the Chroma Vector Store

1. [Langchain Chunking Strategy](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)
2. [Langchain Vectorstore](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)


In [ ]:
# Initialize the gemini-pro language model with specified settings (Change temeprature  and other parameters as per your requirement)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3, safety_settings={
          HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        },)

# Load Documents from a web URL
url = "https://en.wikipedia.org/wiki/Artificial_intelligence"
chunks = load_and_process_data(url)

# Get Embedding Model.
# embedding_model value options (openai, google, huggingface)
embeddings = get_embedding_model()

# Create vector stores using the specified embedding model
vectorstore = create_vector_stores(chunks, embeddings)

# Step 5: RAG in Action to test Embedding Model

## This implementation demonstrashowstes the key parts of Basic RAG with evaluation:

1. Chunking and embedding of source documents
2. Retrieval of relevant documents based on query similarity
3. Generation of a response using the retrieved context
4. Evaluation of the response based on its similarity to both the query and the context

In [ ]:
query = "What are the main applications of artificial intelligence in healthcare?"
result = basic_rag(query, vectorstore, llm)


print(f"Query: {result['query']}")
print("=========================")
print(f"Final Answer: {result['final_answer']}")
print("=========================")
print(f"Retrieval Method: {result['retrieval_method']}")


# Evaluate the response
evaluation = evaluate_response(query, embeddings, result["final_answer"], result["context"])
print("\nEvaluation:")
print(f"Query-Response Similarity: {evaluation['query_response_similarity']:.4f}")
print(f"Response-Context Similarity: {evaluation['response_context_similarity']:.4f}")
print(f"Overall Relevance Score: {evaluation['relevance_score']:.4f}")

Query: What are the main applications of artificial intelligence in healthcare?
Final Answer: The main applications of artificial intelligence in healthcare include increasing patient care and quality of life, more accurately diagnosing and treating patients, processing and integrating big data, and deepening the understanding of biomedically relevant pathways.
Retrieval Method: Basic similarity search

Evaluation:
Query-Response Similarity: 0.9467
Response-Context Similarity: 0.8284
Overall Relevance Score: 0.8876


# Retrieval Demonstration:

We showcase the retrieval process by displaying the retrieved documents for a sample query.

In [ ]:
# Demonstrate retrieval
demo_query = "Explain the concept of machine learning and its relationship to AI"
print(f"\nDemonstration Query: {demo_query}")

# Retrieve documents
docs = vectorstore.similarity_search(demo_query, k=5)
print("\nRetrieved Documents:")
pretty_print_docs(docs)


Demonstration Query: Explain the concept of machine learning and its relationship to AI

Retrieved Documents:
--------------------------------------------------
Document 1:
Content:
Learning
Machine learning is the study of programs that can improve their performance on a given task automatically.[41] It has been a part of AI from the beginning.[e]

Metadata:
  id: 38
  language: en
  source: https://en.wikipedia.org/wiki/Artificial_intelligence
  title: Artificial intelligence - Wikipedia
--------------------------------------------------
Document 2:
Content:
No established unifying theory or paradigm has guided AI research for most of its history.[aa] The unprecedented success of statistical machine learning in the 2010s eclipsed all other approaches (so much so that some sources, especially in the business world, use the term "artificial intelligence" to mean "machine learning with neural networks"). This approach is mostly sub-symbolic, soft and narrow. Critics argue that these qu